In [75]:
# This file contains a writer agent that uses an iterative workflow to improve its content based on the criticism provided by other sub-agents

In [76]:
# Importing libraries
import os
from dotenv import load_dotenv
from google.adk.agents import Agent, LoopAgent, SequentialAgent
from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.tools import FunctionTool
from google.adk.runners import InMemoryRunner


In [77]:
# Checking configuration
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    print("API KEY FOUND")
else:
    print("CONFIGURATION ERROR")

API KEY FOUND


In [78]:
def create_retry_config():
    '''
    The function creates retry config for the Agents
    '''
    return types.HttpRetryOptions(
        attempts=7,
        exp_base=2,
        initial_delay=2,
        http_status_codes=[429, 500, 502, 503, 504]
    )

retry_config = create_retry_config()

In [79]:
# The workflow includes two agents, writer and critic agent, the goal of the writer agent is to pass the criteria of critic agent
initial_writer_agent=Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=''' You are a creative writer. Based on the users prompt write a short story of around (100-150) words.
    Output only the story text, with no introduction or explaination.''',
    description="Writes the initial story draft based on the initial prompt from the user.",
    output_key="current_story"
)
print("Intial agent created")

Intial agent created


In [80]:
critic_agent=Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction='''You are a constuctive story critic. Review the story provided below based on clarity, engagement and basic coherence.
    Story: {current_story},
    
    Evaluate the story's plot, character and pacing.
    - IF the story is well written, reply with the exact phrase "APPROVED",
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.
    ''',
    description="Reviews the current draft, provides critique if clear improvements are needed, otherwise signals completion",
    output_key="critique"
)

print("Critic agent created")

Critic agent created


In [81]:
# This function will be used by the refiner agent to exit the loop
def exit_loop():
    '''Call this function ONLY when the critique is 'APPROVED', indicating the story does not need any further improvements.'''
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

print('Exit loop function created')

Exit loop function created


In [82]:
# The agent refines the story based on the critique or call the exit_loop function

refiner_agent=Agent(
    name="RefinerAgent",
    model=Gemini(
        model='gemini-2.5-flash-lite',
        retry_options=retry_config
    ),
    instruction='''You are a story refiner. You have a story draft and a critique.
    Story: {current_story},
    Critique: {critique}

    Your task is to analyse the critique.
    - IF the critique is EXACTLY "APPROVED", you must call the `exit_loop` function and do nothing.
    - OTHERWISE, rewrite the entire story draft to fully incorporate the critique.''',
    tools=[
        FunctionTool(exit_loop)
    ],
    description="Refines the current draft, if APPROVED call the 'exit_loop' function or else refine the current draft and pass to the critic agent for review",
    output_key="current_story"

)

print("Refiner agent created")

Refiner agent created


In [83]:
story_refinement_loop=LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2 # Helps to prevent infinite loops
)

root_agent=SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("Loop and sequential agent created")

Loop and sequential agent created


In [84]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a horse rider, who fell from the mountain and lost his memory"
)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/arpit/devlopement/agentic-ai-google-kaggle/myvenv/lib/python3.9/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Write a short story about a horse rider, who fell from the mountain and lost his memory
InitialWriterAgent > The wind whipped Elara's hair as she urged her mare, Sol, up the treacherous mountain path. Below, the world fell away in a dizzying expanse of green and grey. A sudden rockslide sent Sol skittering, hooves scrabbling for purchase. Elara fought for control, but gravity claimed them both.

She awoke to the scent of pine and damp earth, the sun dappling her face. A dull ache throbbed in her head, and a blankness where memories should be. Sol was gone. The mountain loomed, a silent witness to her forgotten fall. She rose, a stranger in her own skin, and began to walk, the wind her only guide, the lost past a whisper on the breeze.
CriticAgent > Here's a critique of your story:

**Plot:** The plot is straightforward and effective for a short piece. It establishes a clear inciting incident (the fall) and its consequence (amnesia). T